In [2]:
from keras import layers, models
from keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Input, Activation,Input, Reshape, concatenate, Conv2D, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Dropout, GlobalMaxPooling2D
from keras.regularizers import Regularizer
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from glob import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
 

Using TensorFlow backend.


In [3]:
#이미지 데이터를 32*32*3 사이즈로 불러와 배열에 저장합니다.

from keras.preprocessing import image as krs_image

groups_folder_path = './mixed/train/'
categories = ["Down","Up"]
num_classes = len(categories)
  
X = []
Y = []
  
for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'
  
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            print(image_dir+filename)
            img = krs_image.load_img(image_dir+filename, target_size=(32, 32))
            img = krs_image.img_to_array(img)
            X.append(img/255)
            Y.append(label)

X = np.array(X)
Y = np.array(Y)

./mixed/train/Down/image2490.png
./mixed/train/Down/image2491.png
./mixed/train/Down/image2492.png
./mixed/train/Down/image2493.png
./mixed/train/Down/image2496.png
./mixed/train/Down/image2497.png
./mixed/train/Down/image2498.png
./mixed/train/Down/image2499.png
./mixed/train/Down/image2500.png
./mixed/train/Down/image2501.png
./mixed/train/Down/image2502.png
./mixed/train/Down/image2503.png
./mixed/train/Down/image2504.png
./mixed/train/Down/image2505.png
./mixed/train/Down/image2506.png
./mixed/train/Down/image2508.png
./mixed/train/Down/image2509.png
./mixed/train/Down/image2510.png
./mixed/train/Down/image2511.png
./mixed/train/Down/image2512.png
./mixed/train/Down/image2513.png
./mixed/train/Down/image2514.png
./mixed/train/Down/image2521.png
./mixed/train/Down/image2522.png
./mixed/train/Down/image2523.png
./mixed/train/Down/image2524.png
./mixed/train/Down/image2525.png
./mixed/train/Down/image2543.png
./mixed/train/Down/image2544.png
./mixed/train/Down/image2556.png
./mixed/tr

In [4]:
#Test set도 동일하게 불러옵니다.

groups_folder_path = './mixed/test/'
categories = ["Down","Up"]
num_classes = len(categories)
  
testX = []
testY = []
  
for idex, categorie in enumerate(categories):
    label = [0 for i in range(num_classes)]
    label[idex] = 1
    image_dir = groups_folder_path + categorie + '/'
  
    for top, dir, f in os.walk(image_dir):
        for filename in f:
            print(image_dir+filename)
            img = krs_image.load_img(image_dir+filename, target_size=(32, 32))
            img = krs_image.img_to_array(img)
            testX.append(img/255)
            testY.append(label)

testY = np.array(testY)

./mixed/test/Down/image3970.png
./mixed/test/Down/image3971.png
./mixed/test/Down/image3972.png
./mixed/test/Down/image3974.png
./mixed/test/Down/image3975.png
./mixed/test/Down/image3981.png
./mixed/test/Down/image3983.png
./mixed/test/Down/image3984.png
./mixed/test/Down/image3987.png
./mixed/test/Down/image3997.png
./mixed/test/Down/image3998.png
./mixed/test/Down/image3999.png
./mixed/test/Down/image4000.png
./mixed/test/Down/image4003.png
./mixed/test/Down/image4015.png
./mixed/test/Down/image4016.png
./mixed/test/Down/image4017.png
./mixed/test/Down/image4018.png
./mixed/test/Down/image4019.png
./mixed/test/Down/image4021.png
./mixed/test/Down/image4022.png
./mixed/test/Down/image4023.png
./mixed/test/Down/image4024.png
./mixed/test/Down/image4025.png
./mixed/test/Down/image4026.png
./mixed/test/Down/image4027.png
./mixed/test/Down/image4028.png
./mixed/test/Down/image4029.png
./mixed/test/Down/image4049.png
./mixed/test/Down/image4065.png
./mixed/test/Down/image4068.png
./mixed/

In [5]:
#감성분석 지수를 저장한 csv파일을 불러옵니다.

import random
import pandas as pd
import numpy as np

csv_dir = os.path.join('./mixed')
split='train'
split2='test'
df = pd.read_csv(f'{csv_dir}/{split}.csv', index_col=[0])
df2 = pd.read_csv(f'{csv_dir}/{split2}.csv', index_col=[0])
print(df)
traincsv = df.loc[:,['jp_scoreavg','nk_scoreavg','na_scoreavg','ch_scoreavg']]
testcsv = df2.loc[:,['jp_scoreavg','nk_scoreavg','na_scoreavg','ch_scoreavg']]

     class  jp_scoreavg  nk_scoreavg  na_scoreavg  ch_scoreavg
1     Down     0.675532    -0.140688    -0.174363     0.398096
2     Down     1.101769     1.001680     0.019375     0.691226
3     Down     1.149128     0.841348     0.555382     0.829649
4     Down     1.188595     0.841348     0.490803     0.683084
5     Down     0.225616     0.997672    -0.355184     0.104966
...    ...          ...          ...          ...          ...
1478    Up     0.407161     0.500642     0.251860     0.784865
1479    Up     0.399268     1.262220     0.219570     1.000641
1480    Up     0.351908     0.701057     0.206654     0.617944
1481    Up     0.304549     0.473919     0.245402     0.902931
1482    Up     0.328228     0.313587     0.484345     1.000641

[1482 rows x 5 columns]


In [6]:
#현재 데이터는 Class (Y값) 에 따라 정렬된 데이터이기 때문에 CNN 학습시 학습이 제대로 되지 않을 우려가 있습니다. 이를 방지하기 위해 랜덤하게 순서를 섞어줍니다.

ind = np.random.permutation(len(Y)) 
X= X[ind]
Y= Y[ind]
traincsv = traincsv.iloc[ind]

In [7]:
pic_input = Input(shape=(32, 32, 3))

cnn = Conv2D(128, kernel_size = (3,3), activation='relu')(pic_input)
cnn = Conv2D(100, kernel_size = (3,3), activation='relu',kernel_regularizer=regularizers.l2(0.01))(cnn)
cnn = Dropout(0.3)(cnn)
cnn = Conv2D(80, kernel_size = (3,3), activation='relu',)(cnn)
cnn = MaxPooling2D((2,2))(cnn)
cnn = BatchNormalization()(cnn)
cnn = Conv2D(64, kernel_size = (3,3), activation='relu',kernel_regularizer=regularizers.l2(0.01))(cnn)
cnn = MaxPooling2D((2,2))(cnn)
cnn = Dropout(0.3)(cnn)
cnn = Conv2D(32, kernel_size = (3,3), activation='relu')(cnn)
cnn = GlobalMaxPooling2D()(cnn)
simple_cnn = Model(inputs=pic_input,outputs=cnn)

# L2 regualarizers와 Batch normalization, 0.3 비율의 Dropout을 활용하는 간단한 CNN 모델입니다. 이는 CNN-MLP 모델과 CNN 모델 모두 사용될 것입니다.
# 구조는 다음과 같습니다.

simple_cnn.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 100)       115300    
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 80)        72080     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 80)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 13, 80)        320 

In [8]:
#CNN 레이어들에 이어서 Fully-connected Layers를 연결해줌으로써 이미지데이터 학습에 사용할 CNN 모델을 만듭니다.

x = Dense(128, activation="relu")(simple_cnn.output)
x = layers.BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
x = Dense(2, activation="sigmoid")(x) #출력층은 이진분류를 위한 sigmoid 함수를 사용합니다.
CNN = Model(inputs=simple_cnn.input,outputs=x)


In [9]:
#각 Epochs 별로 val_accuracy가 가장 높은 수치를 갱신할 때마다 모델의 가중치를 저장합니다.

model_path = './mixed/' + '{epoch:02d}-{val_loss:.4f}cnn.hdf5'

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_accuracy', save_best_only=True)

#CNN모델의 손실 함수는 binary_cross-entropy 이며, 최적화기는 Adam을 사용하였습니다.

CNN.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.002), metrics=['accuracy'])

#fit 함수를 통해 CNN 모델을 학습시킬 수 있습니다.

CNN.fit([X],Y, validation_data=([testX],testY),
                         epochs=30, batch_size=10,
                 shuffle=True,
                callbacks=[checkpoint])

Train on 1482 samples, validate on 487 samples
Epoch 1/30
1482/1482 [==============================] - 16s 11ms/step - loss: 1.8194 - accuracy: 0.5135 - val_loss: 1.1652 - val_accuracy: 0.5524
Epoch 2/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.9336 - accuracy: 0.5111 - val_loss: 0.8188 - val_accuracy: 0.5524
Epoch 3/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.8014 - accuracy: 0.5256 - val_loss: 0.7632 - val_accuracy: 0.5524
Epoch 4/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7420 - accuracy: 0.5148 - val_loss: 0.7361 - val_accuracy: 0.5544
Epoch 5/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7454 - accuracy: 0.5108 - val_loss: 0.7425 - val_accuracy: 0.5513
Epoch 6/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7262 - accuracy: 0.5206 - val_loss: 0.7248 - val_accuracy: 0.4990
Epoch 7/30
1482/1482 [==============================] - 15s 10ms/step - loss:

In [11]:
#감성분석지수를 이용한 CNN-MLP 모델

#4D 벡터를 입력값으로 받는 은닉층을 생성합니다.

inputA = Input(shape=(4,))
x = Dense(64, activation="relu")(inputA)
x = Dropout(0.3)(x)
x = Dense(32, activation="relu")(x)
x = Dense(6, activation="relu")(x)
mlp = Model(inputs=inputA, outputs=x)

In [12]:
#mlp를 통과한 값들과 simple_cnn을 통과한 값들이 Concatenated layers를 통해 합쳐지고, 이후 몇 개의 은닉층으 지나 sigmoid 출력값을 반환합니다.

concatenated = layers.concatenate([simple_cnn.output, mlp.output])
concatenated = layers.Dense(32, activation='relu')(concatenated)
concatenated = layers.BatchNormalization()(concatenated)
concatenated = layers.Dense(16, activation='relu')(concatenated)
concat_out = layers.Dense(2, activation='sigmoid')(concatenated)

concat_model = Model([simple_cnn.input, mlp.input], concat_out)

#CNN-MLP 모델의 구조도는 다음과 같습니다.

concat_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 30, 30, 128)  3584        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 100)  115300      conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 28, 28, 100)  0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [13]:
#CNN-MLP model도 동일한 손실 함수와 최적화기를 사용합니다.

concat_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.002), metrics=['accuracy'])

In [12]:
model_path = './mixed/' + '{epoch:02d}-{val_loss:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_accuracy', save_best_only=True)

concat_model.fit([X,traincsv],Y, validation_data=([testX,testcsv],testY),
                         epochs=30, batch_size=10,
                 shuffle=True,
                callbacks=[checkpoint])

Train on 1482 samples, validate on 487 samples
Epoch 1/30
1482/1482 [==============================] - 16s 11ms/step - loss: 0.7683 - accuracy: 0.5121 - val_loss: 0.7282 - val_accuracy: 0.5462
Epoch 2/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7382 - accuracy: 0.5253 - val_loss: 0.7624 - val_accuracy: 0.5318
Epoch 3/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7399 - accuracy: 0.5321 - val_loss: 0.7380 - val_accuracy: 0.5339
Epoch 4/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7293 - accuracy: 0.5337 - val_loss: 0.7383 - val_accuracy: 0.5503
Epoch 5/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7537 - accuracy: 0.5351 - val_loss: 1.0224 - val_accuracy: 0.4476
Epoch 6/30
1482/1482 [==============================] - 15s 10ms/step - loss: 0.7491 - accuracy: 0.5331 - val_loss: 0.7432 - val_accuracy: 0.5411
Epoch 7/30
1482/1482 [==============================] - 15s 10ms/step - loss:

In [16]:
#CNN모델과 CNN-MLP모델은 아래와 같이 json 파일로 저장하였습니다.

concatm =concat_model.to_json()
cnnm = CNN.to_json()

with open("CNNMLPmodel.json", "w") as json_file : 
    json_file.write(concatm)
with open("CNNmodel.json", "w") as json_file : 
    json_file.write(cnnm)

In [53]:
#아래 과정을 통해서 저장된 가중치 값과 모델 json 파일을 불러와 Accuracy, F1_score 값과 Confusion matrix를 확인할 수 있습니다. 

from keras.models import model_from_json
json_file = open("CNNMLPmodel.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('0.5852.hdf5')
loaded_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.002), metrics=['accuracy'])
scores = loaded_model.evaluate([testX,testcsv],testY)
print("%s: %.2f%%" %(loaded_model.metrics_names[1], scores[1]*100))

output = loaded_model.predict([testX,testcsv])
print(output)
y_pred = np.argmax(output, axis=1)
print(y_pred)
y_true = np.argmax(testY, axis=1)
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

f1score = f1_score(y_true, y_pred)
print(f1score)
x = classification_report(y_true, y_pred)
print(x)
cm = confusion_matrix(y_true, y_pred)
print(cm)

487/487 [==============================] - 1s 2ms/step
accuracy: 58.52%
[[0.4319516  0.62624985]
 [0.4124375  0.6272991 ]
 [0.44581762 0.6150354 ]
 [0.46909422 0.54702115]
 [0.46204352 0.54564494]
 [0.5017508  0.5159557 ]
 [0.490478   0.5528849 ]
 [0.42474523 0.5940024 ]
 [0.47902608 0.5780092 ]
 [0.46208733 0.5385783 ]
 [0.47057605 0.5545722 ]
 [0.46267495 0.59056944]
 [0.47072002 0.59312665]
 [0.46473208 0.57592374]
 [0.48123717 0.5766964 ]
 [0.4656095  0.61285037]
 [0.38082826 0.6375884 ]
 [0.3682658  0.6389847 ]
 [0.40274605 0.6402645 ]
 [0.44954023 0.5884976 ]
 [0.45763204 0.5816858 ]
 [0.42370164 0.6311321 ]
 [0.42072973 0.6451115 ]
 [0.45808998 0.5893336 ]
 [0.47612292 0.5872933 ]
 [0.45354125 0.62047505]
 [0.4419751  0.6015502 ]
 [0.44195333 0.5975058 ]
 [0.459988   0.56009287]
 [0.44818142 0.5935217 ]
 [0.43311194 0.5964777 ]
 [0.46181458 0.58603567]
 [0.44751284 0.57894886]
 [0.47191665 0.5782704 ]
 [0.4499444  0.5822819 ]
 [0.45751747 0.6255564 ]
 [0.46152177 0.61168885]
 [0